In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../mnist/data/", one_hot=True)


total_epoch = 100
batch_size = 100
learning_rate = 0.0002

n_hidden = 256
n_input = 28 * 28
n_noise = 128


X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])



G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))


D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))


def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)

    return output


def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)

    return output


def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))


G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)


loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))


D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]


train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

#         _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
#         _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
        _, _, loss_val_D, loss_val_G = sess.run([train_D, train_G, loss_D, loss_G], feed_dict={X: batch_xs, Z: noise})

    print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))


    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')

Extracting ../mnist/data/train-images-idx3-ubyte.gz
Extracting ../mnist/data/train-labels-idx1-ubyte.gz
Extracting ../mnist/data/t10k-images-idx3-ubyte.gz
Extracting ../mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0000 D loss: -0.3781 G loss: -2.038
Epoch: 0001 D loss: -0.2113 G loss: -2.47
Epoch: 0002 D loss: -0.1176 G loss: -3.154
Epoch: 0003 D loss: -0.3057 G loss: -1.947
Epoch: 0004 D loss: -0.3279 G loss: -2.157
Epoch: 0005 D loss: -0.1139 G loss: -3.415
Epoch: 0006 D loss: -0.1421 G loss: -2.662
Epoch: 0007 D loss: -0.142 G loss: -2.851
Epoch: 0008 D loss: -0.07649 G loss: -3.332
Epoch: 0009 D loss: -0.1579 G loss: -3.064
Epoch: 0010 D loss: -0.1542 G loss: -3.08
Epoch: 0011 D loss: -0.3497 G loss: -2.813
Epoch: 0012 D loss: -0.3016 G loss: -2.682
Epoch: 0013 D loss: -0.5142 G loss: -2.438
Epoch: 0014 D loss: -0.3337 G loss: -2.628
Epoch: 0015 D loss: -0.5548 G loss: -2.314
Epoch: 0016 D loss: -0.3155 G loss: -2.572
Epoch: 0017 D loss: -0.2554 G loss: -2.493
Epoch: 0018 D loss: -0

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../mnist/data/", one_hot=True)


total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation = tf.nn.relu )
        output = tf.layers.dense(hidden, n_input activation = tf.nn.sigmoid)
    return output

def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator'): as scope:
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation = tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation = None)
    return output